Lambda School Data Science

*Unit 4, Sprint 1, Module 4*

---

# Topic Modeling (Prepare)

On Monday we talked about summarizing your documents using just token counts. Today, we're going to learn about a much more sophisticated approach - learning 'topics' from documents. Topics are a latent structure. They are not directly observable in the data, but we know they're there by reading them.

> **latent**: existing but not yet developed or manifest; hidden or concealed.

## Use Cases
Primary use case: what the hell are your documents about? Who might want to know that in industry - 
* Identifying common themes in customer reviews
* Discovering the needle in a haystack 
* Monitoring communications (Email - State Department) 

## Learning Objectives
*At the end of the lesson you should be able to:*
* <a href="#p1">Part 1</a>: Describe how an LDA Model works
* <a href="#p2">Part 2</a>: Estimate a LDA Model with Gensim
* <a href="#p3">Part 3</a>: Interpret LDA results
* <a href="#p4">Part 4</a>: Select the appropriate number of topics


# Latent Dirchilet Allocation Models (Learn)
<a id="#p1"></a>

## Overview
LDA is a "generative probabilistic model". 

Let's play with a model available [here](https://lettier.com/projects/lda-topic-modeling/)

## Follow Along

## Challenge 

# Estimating LDA Models with Gensim (Learn)
<a id="#p1"></a>

## Overview
### A Literary Introduction: *Jane Austen V. Charlotte Bronte*
Despite being born nearly forty years apart, modern fans often pit Jane Austen & Charlotte Bronte against one another in a battle for literary  supremacy. The battle centers around the topics of education for women, courting, and marriage. The authors' similar backgrounds naturally draw comparisons, but the modern fascination is probably due to novelty of British women publishing novels during the early 19th century. 

Can we help close a literary battle for supremacy and simply acknowledge that the authors addressed different topics and deserve to be acknowledged as excellent authors each in their own right?

We're going to apply Latent Dirichlet Allocation a machine learning algorithm for topic modeling to each of the author's novels to compare the distribution of topics in their novels.

In [46]:
import numpy as np
import gensim
import os
import re
from ast import literal_eval
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore
import tqdm
import pandas as pd

In [2]:
gensim.__version__

'3.8.1'

### Novel Data
I grabbed the novel data pre-split into a bunch of smaller chunks

In [52]:
path = './data/imbd_keywords.csv'
df = pd.read_csv(path)
print(df.shape)
df.head()

(40436, 3)


,review,sentiment,keywords
0,One of the other reviewers has mentioned that ...,positive,"['other shows', 'graphic violence', 'prison ex..."
1,A wonderful little production. The filming tec...,positive,"['halliwell', 'michael sheen', 'realism', 'com..."
2,I thought this was a wonderful way to spend ti...,positive,"['spirited young woman', 'devil wears prada', ..."
3,Basically there's a family where a little boy ...,negative,"['playing parents', 'jake', 'parents', 'descen..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"['mr. mattei', 'good luck', 'mattei', 'human r..."


In [53]:
df['keywords']= df['keywords'].apply(literal_eval)

In [59]:
df.head()

,review,sentiment,keywords
0,One of the other reviewers has mentioned that ...,positive,"[other shows, graphic violence, prison experie..."
1,A wonderful little production. The filming tec...,positive,"[halliwell, michael sheen, realism, comedy, wi..."
2,I thought this was a wonderful way to spend ti...,positive,"[spirited young woman, devil wears prada, summ..."
3,Basically there's a family where a little boy ...,negative,"[playing parents, jake, parents, descent dialo..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[mr. mattei, good luck, mattei, human relation..."


In [87]:
list(df)

['review', 'sentiment', 'keywords']

In [71]:
stop_words = {
    'movie',
    'review',
    'film',
    'actor',
    'actress',
    'films'
}
import spacy
nlp =spacy.load('en_core_web_lg')

In [72]:
stop = nlp.Defaults.stop_words.union(stop_words)

In [73]:
# Get a sense of whats in the keywords
df['keywords'][0]

['other shows',
 'graphic violence',
 'prison experience',
 'prison bitches',
 'violence',
 'emerald city',
 'dodgy dealings',
 'glass fronts',
 'inmates',
 'street skills',
 'unflinching scenes',
 'crooked guards',
 'the oswald maximum security state penitentary',
 'mainstream audiences',
 'oz',
 'city',
 'shady agreements',
 'pretty pictures',
 'touch',
 'muslims',
 'irish',
 'italians',
 'gangstas',
 'latinos',
 'christians',
 'home',
 'first',
 'charm',
 'the high levels',
 'drugs',
 'right',
 'regards',
 'well mannered, middle class inmates',
 'sex',
 'a show',
 'scuffles',
 'order',
 'aryans',
 'just 1 oz episode',
 'the first episode',
 'the other reviewers',
 'your darker side',
 'the prison',
 'the main appeal',
 'romance',
 'the first thing',
 'an experimental section',
 'privacy',
 'the classic use',
 'the word',
 'no punches',
 'its brutality',
 'their lack',
 'a taste',
 'the fact',
 'the nickname',
 'the agenda',
 'all the cells',
 'a nickel',
 'you']

In [ ]:
split_list =[i.split() for i in l]

In [91]:
def remove_stops(_list_):
    
    keywords = []
    
    for keyword in _list_:
        
        phrase = []
        
        words = keyword.split()
        
        for word in words:
            if word in stop:
                pass
            else:
                phrase.append(word)
        phrase = ' '.join(phrase)
        
        if len(phrase) > 0:
            keywords.append(phrase)
        
    return keywords

In [92]:
remove_stops(df['keywords'][0])

['shows',
 'graphic violence',
 'prison experience',
 'prison bitches',
 'violence',
 'emerald city',
 'dodgy dealings',
 'glass fronts',
 'inmates',
 'street skills',
 'unflinching scenes',
 'crooked guards',
 'oswald maximum security state penitentary',
 'mainstream audiences',
 'oz',
 'city',
 'shady agreements',
 'pretty pictures',
 'touch',
 'muslims',
 'irish',
 'italians',
 'gangstas',
 'latinos',
 'christians',
 'home',
 'charm',
 'high levels',
 'drugs',
 'right',
 'regards',
 'mannered, middle class inmates',
 'sex',
 'scuffles',
 'order',
 'aryans',
 '1 oz episode',
 'episode',
 'reviewers',
 'darker',
 'prison',
 'main appeal',
 'romance',
 'thing',
 'experimental section',
 'privacy',
 'classic use',
 'word',
 'punches',
 'brutality',
 'lack',
 'taste',
 'fact',
 'nickname',
 'agenda',
 'cells',
 'nickel']

In [94]:
from tqdm import tqdm

tqdm.pandas()

df['clean_keywords'] = df['keywords'].progress_apply(remove_stops)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 40436/40436 [00:01<00:00, 21429.50it/s]


In [95]:
import gensim
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

In [96]:
# Dict representation of all our keywords
id2word = corpora.Dictionary(df['clean_keywords'])

In [97]:
id2word.token2id['sea-monster']

18960

In [98]:
len(id2word.keys())

491029

In [122]:
id2word.filter_extremes(no_below=5, no_above=0.85)

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [123]:
len(id2word.keys())

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


29754

In [124]:
corpus = [id2word.doc2bow(text) for text in df['clean_keywords']]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [125]:
corpus[812]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(77, 1),
 (585, 1),
 (674, 1),
 (809, 1),
 (1191, 1),
 (2226, 1),
 (3761, 1),
 (3969, 1),
 (5216, 1),
 (5559, 1),
 (8427, 1)]

In [126]:
id2word[77]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'time'

In [128]:
lda = LdaMulticore(corpus=corpus,
                  id2word=id2word,
                  num_topics=20,
                  passes=50,
                  workers=2
                  )

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [129]:
lda.print_topics()

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.010*"father" + 0.009*"love" + 0.008*"man" + 0.008*"life" + 0.008*"story" + 0.007*"american" + 0.007*"wife" + 0.007*"son" + 0.006*"mother" + 0.006*"scene"'),
 (1,
  '0.008*"time" + 0.006*"cartoon" + 0.005*"success" + 0.005*"tv" + 0.004*"mars" + 0.004*"television" + 0.004*"nose" + 0.003*"cartoons" + 0.003*"video" + 0.003*"great acting"'),
 (2,
  '0.021*"british" + 0.009*"american" + 0.008*"time" + 0.008*"australian" + 0.006*"oscar" + 0.006*"london" + 0.006*"irish" + 0.006*"england" + 0.005*"hollywood" + 0.004*"cast"'),
 (3,
  '0.008*"story" + 0.006*"man" + 0.005*"hitchcock" + 0.005*"music" + 0.004*"hero" + 0.004*"role" + 0.004*"action" + 0.004*"wife" + 0.004*"town" + 0.004*"character"'),
 (4,
  '0.027*"story" + 0.021*"acting" + 0.018*"actors" + 0.015*"characters" + 0.012*"book" + 0.011*"character" + 0.010*"role" + 0.010*"great" + 0.009*"director" + 0.009*"people"'),
 (5,
  '0.015*"life" + 0.012*"people" + 0.011*"love" + 0.010*"way" + 0.010*"story" + 0.008*"world" + 0.008*"end" 

In [130]:
import re 

words = [re.findall(r'"[^"]*"', t[1]) for t in lda.print_topics(20)]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [131]:
topics = [', '.join(t[0:5]) for t in words]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [132]:
for id, t in enumerate(topics):
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
"father", "love", "man", "life", "story"

------ Topic 1 ------
"time", "cartoon", "success", "tv", "mars"

------ Topic 2 ------
"british", "american", "time", "australian", "oscar"

------ Topic 3 ------
"story", "man", "hitchcock", "music", "hero"

------ Topic 4 ------
"story", "acting", "actors", "characters", "book"

------ Topic 5 ------
"life", "people", "love", "way", "story"

------ Topic 6 ------
"earth", "sci-fi", "star trek", "plot", "oz"

------ Topic 7 ------
"time", "people", "american", "hollywood", "fact"

------ Topic 8 ------
"time", "characters", "story", "people", "plot"

------ Topic 9 ------
"guy", "night", "company", "friends", "time"

------ Topic 10 ------
"year", "dvd", "character", "batman", "story"

------ Topic 11 ------
"time", "people", "story", "world", "life"

------ Topic 12 ------
"christian", "people", "time", "end", "jesus"

------ Topic 13 ------
"music", "role", "plot", "version", "dvd"

------ Topic 14 ------
"kids", "time

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [133]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [134]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.164046  0.010512       1        1  15.117847
11    -0.116836  0.034786       2        1   9.190754
5     -0.120310  0.113962       3        1   7.452006
0     -0.017223  0.064687       4        1   6.117090
8     -0.168491 -0.017711       5        1   6.109539
14    -0.091270  0.077760       6        1   5.723546
4     -0.170911 -0.125261       7        1   5.413027
15    -0.104039 -0.044562       8        1   4.994263
7     -0.051539 -0.030051       9        1   4.809009
18    -0.038689  0.141834      10        1   4.752122
3      0.043825 -0.026918      11        1   4.262010
19    -0.039282 -0.099801      12        1   3.976119
16     0.058989  0.202015      13        1   3.757159
13     0.009930 -0.186730      14        1   3.452337
2      0.104013 -0.011594      15        1   2.996616
12     0.131576  0.064494      16        1   2.676143
10     0.074955 -0.194736      17        1   2.622548
9      0.179847  0.169774      18        1   2.376039
6      0.252467 -0.041508      19        1   2.110430
1      0.227034 -0.100955      20        1   2.091397, topic_info=        Term         Freq        Total Category  logprob  loglift
94    acting  4289.000000  4289.000000  Default  30.0000  30.0000
133     kids  1471.000000  1471.000000  Default  29.0000  29.0000
39    actors  3413.000000  3413.000000  Default  28.0000  28.0000
10   episode  1048.000000  1048.000000  Default  27.0000  27.0000
138    story  8264.000000  8264.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
279     lots    55.521863  1070.069775  Topic20  -5.9767   0.9086
659     days    54.567341  1137.794028  Topic20  -5.9941   0.8299
11      fact    55.640682  4458.330153  Topic20  -5.9746  -0.5163
98      case    50.819973  1829.413299  Topic20  -6.0652   0.2839
54    scenes    50.803016  2695.206904  Topic20  -6.0655  -0.1039

[1469 rows x 6 columns], token_table=       Topic      Freq             Term
term                                   
12898     17  0.974726         "horror"
5342      16  0.975137                1
8982       1  0.093116            1970s
8982       9  0.356943            1970s
8982      13  0.067250            1970s
...      ...       ...              ...
4301       5  0.207683      young woman
4301      14  0.059338      young woman
10023      3  0.984548  younger brother
93         1  0.991249           zombie
3932       1  0.997941          zombies

[3565 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 12, 6, 1, 9, 15, 5, 16, 8, 19, 4, 20, 17, 14, 3, 13, 11, 10, 7, 2])

In [147]:
distro = [lda[d] for d in corpus]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [148]:
distro[0]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 0.05985297),
 (5, 0.19525698),
 (6, 0.062890366),
 (7, 0.09723046),
 (12, 0.10100644),
 (13, 0.1630213),
 (17, 0.15018634),
 (18, 0.072049685),
 (19, 0.08470704)]

In [159]:
distro = [lda[d] for d in corpus]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [160]:
def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist

new_distro = [update(d) for d in distro]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [161]:
len(new_distro)

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


40436

In [162]:
df.head()

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,review,sentiment,keywords,clean_keywords
0,One of the other reviewers has mentioned that ...,positive,"[other shows, graphic violence, prison experie...","[shows, graphic violence, prison experience, p..."
1,A wonderful little production. The filming tec...,positive,"[halliwell, michael sheen, realism, comedy, wi...","[halliwell, michael sheen, realism, comedy, wi..."
2,I thought this was a wonderful way to spend ti...,positive,"[spirited young woman, devil wears prada, summ...","[spirited young woman, devil wears prada, summ..."
3,Basically there's a family where a little boy ...,negative,"[playing parents, jake, parents, descent dialo...","[playing parents, jake, parents, descent dialo..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[mr. mattei, good luck, mattei, human relation...","[mr. mattei, good luck, mattei, human relation..."


In [163]:
df = pd.DataFrame.from_records(new_distro,index=titles)
df.columns = topics

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'titles' is not defined

In [ ]:
topics.head()

In [ ]:
df['primary_topic'] = topics.idmax{axis=1}

In [ ]:
df['primary_topic'].value_counts()

In [ ]:
agg = pd.pivot_table(df, values=['sentiment'],
                    index=['primary_topic'],
                    
                    )

In [ ]:
agg = agg.reset_index()

In [ ]:
import seaborm as sns
import matplotlib.pyplot as plt
sns.set_color_codes('pastel')
sns.barplot(x='negative', y='primary_topic', data=agg, label='Negative', color='b')

sns.set_color_codes('muted')
sns.barplot(x='postive', y='primary_topic', data=agg, label='Postive', color='b')

ax.legend(ncol=2, loc='lower right', frameon=True)
ax.set(xlim=(0,5000), ylabel='')

### Text Preprocessing
**Activity**: update the function `tokenize` with any technique you have learned so far this week. 

In [17]:
# 1) Plain Python - ''.split command
# 2) Spacy - just the lemmas from the document
# 3) Gensim - simple_preprocess

STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [18]:
import os

def gather_data(path_to_data): 
    data = []
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    data.append(tokenize(str(text)))       
    return data

In [19]:
tokens = gather_data(path)

NotADirectoryError: [WinError 267] The directory name is invalid: './data/imbd_keywords.csv'

In [15]:
tokens[0][0:10]

['giving',
 'fair',
 'companion',
 'account',
 'yesterday',
 'party',
 'friend',
 'cole',
 'come',
 'stilton']

In [16]:
"this is a sample string with a \n newline character".replace('\n', '')

'this is a sample string with a  newline character'

## Follow Along

In [17]:
titles = [t[:-4] for t in os.listdir(path) if os.path.isdir(t) == False]

In [18]:
len(titles)

813

In [20]:
len(tokens)

813

### Author DataFrame


In [21]:
df = pd.DataFrame(index=titles, data={'tokens':tokens})

In [22]:
df.head()

,tokens
Austen_Emma0026,"[giving, fair, companion, account, yesterday, ..."
Austen_Emma0032,"[feels, like, snow, place, party, try, day, di..."
CBronte_Villette0086,"[pierced, opaque, blackness, stood, bougie, qu..."
CBronte_Jane0099,"[joke, queer, looks, tell, thing, specially, s..."
CBronte_Villette0092,"[second, landing, floor, comprising, abode, kn..."


In [23]:
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df.reset_index()['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df.reset_index()['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [24]:
df['author'] = df['author'].map({'Austen':1, 'CBronte':0})

In [25]:
df.author.value_counts()

0    441
1    372
Name: author, dtype: int64

In [26]:
df.head()

,tokens,author,book,section
Austen_Emma0026,"[giving, fair, companion, account, yesterday, ...",1,Emma,26
Austen_Emma0032,"[feels, like, snow, place, party, try, day, di...",1,Emma,32
CBronte_Villette0086,"[pierced, opaque, blackness, stood, bougie, qu...",0,Villette,86
CBronte_Jane0099,"[joke, queer, looks, tell, thing, specially, s...",0,Jane,99
CBronte_Villette0092,"[second, landing, floor, comprising, abode, kn...",0,Villette,92


### Streaming Documents
Here we use a new pythonic thingy: the `yield` statement in our function. This allows us to iterate over a bunch of documents without actually reading them into memory. You can see how we use this function later on. 

In [27]:
def doc_stream(path):
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    tokens = tokenize(text)
                yield tokens

In [28]:
streaming_data = doc_stream(path)

In [29]:
type(streaming_data)

generator

In [ ]:
# gather_data => returns a list
# doc_stream => returns a generator

In [30]:
next(streaming_data) # Returns one document at a time from the generator

['giving',
 'fair',
 'companion',
 'account',
 'yesterday',
 'party',
 'friend',
 'cole',
 'come',
 'stilton',
 'cheese',
 'north',
 'wiltshire',
 'butter',
 'cellery',
 'beet',
 'root',
 'dessert',
 'soon',
 'led',
 'better',
 'course',
 'consoling',
 'reflection',
 'thing',
 'interests',
 'love',
 'thing',
 'serve',
 'introduction',
 'near',
 'heart',
 'kept',
 'longer',
 'away',
 'walked',
 'quietly',
 'till',
 'view',
 'vicarage',
 'pales',
 'sudden',
 'resolution',
 'getting',
 'harriet',
 'house',
 'amiss',
 'boot',
 'fall',
 'arrange',
 'broke',
 'lace',
 'short',
 'dexterously',
 'throwing',
 'ditch',
 'presently',
 'obliged',
 'entreat',
 'stop',
 'acknowledged',
 'inability',
 'rights',
 'able',
 'walk',
 'home',
 'tolerable',
 'comfort',
 'lace',
 'gone',
 'know',
 'contrive',
 'troublesome',
 'companion',
 'hope',
 'ill',
 'equipped',
 'elton',
 'beg',
 'leave',
 'stop',
 'house',
 'ask',
 'housekeeper',
 'bit',
 'ribband',
 'string',
 'thing',
 'boot',
 'elton',
 'looked',

### Gensim LDA Topic Modeling

In [31]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(doc_stream(path))

In [32]:
id2word.token2id['england']

869

In [33]:
id2word.doc2bow(tokenize("This is a sample message Darcy England England England"))

[(869, 3), (1254, 1), (2485, 1), (16850, 1)]

In [36]:
import sys
print(sys.getsizeof(id2word))
print(sys.getsizeof(tokens))

56
7056


In [37]:
len(id2word.keys())

22095

In [38]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=0.95)

In [39]:
len(id2word.keys())

8102

In [40]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
# Although abstracted away - tokenization IS happening in the doc_stream f(x)
corpus = [id2word.doc2bow(text) for text in doc_stream(path)]

In [41]:
corpus[345][:10]

[(4, 1),
 (17, 1),
 (28, 1),
 (30, 1),
 (31, 2),
 (40, 1),
 (46, 1),
 (51, 1),
 (55, 1),
 (71, 1)]

In [42]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )

In [65]:
lda.print_topics()

[(0,
  '0.009*"pounds" + 0.007*"mother" + 0.006*"sure" + 0.006*"think" + 0.006*"year" + 0.004*"rochester" + 0.004*"certainly" + 0.004*"thornfield" + 0.004*"fairfax" + 0.004*"live"'),
 (1,
  '0.008*"pelet" + 0.005*"thought" + 0.005*"like" + 0.004*"little" + 0.004*"man" + 0.004*"hunsden" + 0.004*"time" + 0.003*"having" + 0.003*"pupils" + 0.003*"crimsworth"'),
 (2,
  '0.008*"madame" + 0.006*"little" + 0.006*"like" + 0.004*"day" + 0.004*"vous" + 0.004*"thought" + 0.003*"know" + 0.003*"beck" + 0.003*"good" + 0.003*"paul"'),
 (3,
  '0.005*"night" + 0.005*"look" + 0.005*"like" + 0.004*"old" + 0.004*"est" + 0.004*"looked" + 0.003*"sir" + 0.003*"thought" + 0.003*"house" + 0.003*"hunsden"'),
 (4,
  '0.007*"little" + 0.005*"madame" + 0.005*"like" + 0.004*"thought" + 0.004*"know" + 0.004*"hand" + 0.004*"good" + 0.004*"night" + 0.003*"day" + 0.003*"dr"'),
 (5,
  '0.017*"elizabeth" + 0.013*"darcy" + 0.010*"miss" + 0.010*"bingley" + 0.010*"bennet" + 0.009*"jane" + 0.007*"know" + 0.007*"think" + 0.006

In [43]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [44]:
topics = [' '.join(t[0:5]) for t in words]

In [45]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
pounds mother sure think year

------ Topic 1 ------
pelet thought like little man

------ Topic 2 ------
madame little like day vous

------ Topic 3 ------
night look like old est

------ Topic 4 ------
little madame like thought know

------ Topic 5 ------
elizabeth darcy miss bingley bennet

------ Topic 6 ------
know jane life like long

------ Topic 7 ------
frances monsieur hunsden little english

------ Topic 8 ------
good little like emma elton

------ Topic 9 ------
sir long jane rochester little

------ Topic 10 ------
like little rochester miss room

------ Topic 11 ------
emma miss harriet thing weston

------ Topic 12 ------
jane emma little thing miss

------ Topic 13 ------
elinor marianne sister mother time

------ Topic 14 ------
monsieur henri georgiana mdlle little



## Challenge 

You will apply an LDA model to a customer review dataset to practice the fitting and estimation of LDA. 

# Interpret LDA Results (Learn)
<a id="#p3"></a>

## Overview

## Follow Along

### Topic Distance Visualization

In [46]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [47]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS10/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.045594 -0.020555       1        1  21.411949
4     -0.077666 -0.055242       2        1  17.607441
11     0.141288 -0.008635       3        1  16.260614
13     0.104042 -0.014424       4        1  16.003267
5      0.152210 -0.012565       5        1  10.043391
6     -0.013861 -0.051117       6        1   6.805909
2     -0.077657 -0.072669       7        1   3.625784
1     -0.033810 -0.047089       8        1   2.328232
7     -0.071731 -0.053929       9        1   1.555922
12     0.095366  0.031604      10        1   1.282463
8      0.052507  0.003884      11        1   1.224821
9      0.006462  0.050561      12        1   0.866362
3     -0.099428  0.027440      13        1   0.458783
0     -0.046036  0.148620      14        1   0.262592
14    -0.086093  0.074117      15        1   0.262464, topic_info=     Category         Freq          Term        Total  loglift  logprob
90    Default   836.000000          emma   836.000000  30.0000  30.0000
177   Default  1711.000000        little  1711.000000  29.0000  29.0000
848   Default   931.000000          jane   931.000000  28.0000  28.0000
1131  Default  1535.000000          miss  1535.000000  27.0000  27.0000
414   Default  1318.000000          like  1318.000000  26.0000  26.0000
495   Default  1312.000000         think  1312.000000  25.0000  25.0000
1641  Default   657.000000     elizabeth   657.000000  24.0000  24.0000
1197  Default   596.000000           sir   596.000000  23.0000  23.0000
387   Default  1349.000000          good  1349.000000  22.0000  22.0000
271   Default   776.000000         thing   776.000000  21.0000  21.0000
169   Default  1437.000000          know  1437.000000  20.0000  20.0000
913   Default   357.000000     rochester   357.000000  19.0000  19.0000
195   Default   672.000000        mother   672.000000  18.0000  18.0000
272   Default  1136.000000       thought  1136.000000  17.0000  17.0000
681   Default   920.000000         shall   920.000000  16.0000  16.0000
134   Default   510.000000       harriet   510.000000  15.0000  15.0000
1297  Default   361.000000       fairfax   361.000000  14.0000  14.0000
2119  Default   424.000000         darcy   424.000000  13.0000  13.0000
702   Default   614.000000          sure   614.000000  12.0000  12.0000
89    Default   370.000000         elton   370.000000  11.0000  11.0000
496   Default  1315.000000          time  1315.000000  10.0000  10.0000
2721  Default   695.000000        elinor   695.000000   9.0000   9.0000
2745  Default   576.000000      marianne   576.000000   8.0000   8.0000
179   Default   934.000000          long   934.000000   7.0000   7.0000
509   Default   427.000000        weston   427.000000   6.0000   6.0000
351   Default  1058.000000           day  1058.000000   5.0000   5.0000
562   Default   587.000000          dear   587.000000   4.0000   4.0000
434   Default   622.000000         night   622.000000   3.0000   3.0000
2485  Default   272.000000      monsieur   272.000000   2.0000   2.0000
148   Default   753.000000         house   753.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
2198  Topic15     1.190997      capacity    21.045567   3.0709  -6.5377
1831  Topic15     2.633591  mademoiselle   114.244537   2.1728  -5.7442
1653  Topic15     1.194079      firmness    22.450714   3.0089  -6.5352
1906  Topic15     1.775551        accent    53.255627   2.5418  -6.1384
5422  Topic15     1.162460         hints    21.797037   3.0116  -6.5620
6008  Topic15     1.547669      pictures    41.826523   2.6460  -6.2758
1251  Topic15     2.444013          book   166.297699   1.7227  -5.8189
1640  Topic15     1.620731         eliza    60.185520   2.3283  -6.2297
2087  Topic15     1.125723        volume    23.958691   2.8849  -6.5941
1138  Topic15     2.362272       natural   183.846130   1.5883  -5.8529
318   Topic15  

### Overall Model / Documents

In [71]:
lda[corpus[0]]

[(10, 0.1378439), (11, 0.77550125), (13, 0.084552065)]

In [72]:
distro = [lda[d] for d in corpus]

In [73]:
distro[0]

[(10, 0.13785224), (11, 0.7755348), (13, 0.08451018)]

In [74]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [75]:
len(new_distro)

813

In [83]:
df.head()

,tokens,author,book,section
Austen_Emma0026,"[she, found, that, he, was, only, his, an, of,...",1,Emma,26
Austen_Emma0032,"[so, very, much, that, if, it, were, to, any, ...",1,Emma,32
CBronte_Villette0086,"[the, there, the, on, the, but, where, was, th...",0,Villette,86
CBronte_Jane0099,"[by, and, one, of, his, and, she, could, not, ...",0,Jane,99
CBronte_Villette0092,"[on, the, the, first, the, of, not, what, as, ...",0,Villette,92


In [84]:
df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [85]:
df.head()

,pounds mother sure think year,pelet thought like little man,madame little like day vous,night look like old est,little madame like thought know,elizabeth darcy miss bingley bennet,know jane life like long,frances monsieur hunsden little english,good little like emma elton,sir long jane rochester little,like little rochester miss room,emma miss harriet thing weston,jane emma little thing miss,elinor marianne sister mother time,monsieur henri georgiana mdlle little,author
Austen_Emma0026,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.137811,0.775370,0.0,0.084716,0.0,Austen
Austen_Emma0032,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.997455,0.0,0.000000,0.0,Austen
CBronte_Villette0086,0.0,0.0,0.0,0.0,0.215021,0.000000,0.782767,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,CBronte
CBronte_Jane0099,0.0,0.0,0.0,0.0,0.000000,0.000000,0.522731,0.0,0.0,0.0,0.473158,0.000000,0.0,0.000000,0.0,CBronte
CBronte_Villette0092,0.0,0.0,0.0,0.0,0.561907,0.027239,0.000000,0.0,0.0,0.0,0.370954,0.038246,0.0,0.000000,0.0,CBronte


In [86]:
df.groupby('author').mean()

,pounds mother sure think year,pelet thought like little man,madame little like day vous,night look like old est,little madame like thought know,elizabeth darcy miss bingley bennet,know jane life like long,frances monsieur hunsden little english,good little like emma elton,sir long jane rochester little,like little rochester miss room,emma miss harriet thing weston,jane emma little thing miss,elinor marianne sister mother time,monsieur henri georgiana mdlle little
author,,,,,,,,,,,,,,,
Austen,0.002680,0.005264,0.000558,0.000027,0.002224,0.229066,0.005129,0.000833,0.016319,0.005213,0.014229,0.352142,0.023068,0.340478,0.000000
CBronte,0.002808,0.035885,0.061155,0.008203,0.298077,0.005798,0.123299,0.026866,0.007873,0.012273,0.385654,0.005757,0.004656,0.014485,0.004485


## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text preprocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow


# Selecting the Number of Topics (Learn)
<a id="#p4"></a>

## Overview

## Follow Along

In [87]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=40, 
                                                        step=2,
                                                        passes=100)

In [89]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
topic_coherence.head()

In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
lda[id2word.doc2bow(tokenize("This is a sample document to score with a topic distribution."))]

# Sources

### *References*
* [Andrew Ng et al paper on LDA](https://ai.stanford.edu/~ang/papers/jair03-lda.pdf)
* On [Coherence](https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf)

### *Resources*

* [Gensim](https://radimrehurek.com/gensim/): Python package for topic modeling, nlp, word vectorization, and few other things. Well maintained and well documented.
* [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): A kind of cookbook for LDA with gensim. Excellent overview, but the you need to be aware of missing import statements and assumed prior knowledge.
* [Chinese Restuarant Process](https://en.wikipedia.org/wiki/Chinese_restaurant_process): That really obscure stats thing I mentioned... 
* [PyLDAvis](https://github.com/bmabey/pyLDAvis): Library for visualizing the topic model and performing some exploratory work. Works well. Has a direct parrell implementation in R as well. 
* [Rare Technologies](https://rare-technologies.com/): The people that made & maintain gensim and a few other libraries.
* [Jane Austen v. Charlotte Bronte](https://www.literaryladiesguide.com/literary-musings/jane-austen-charlotte-bronte-different-alike/)